# Разбор списка ФСЭМ

Миньюс РФ ведёт [список экстремистских материалов](http://minjust.ru/ru/extremist-materials), которые должны быть заблокированы на территории России. 
![](thumb.png)

Список это несколько тысяч записей вида:
```
Музыкальный альбом "Музыка белых", автор - Музыкальная группа Order

Аудиофайл – «Русский стяг – Я не хочу быть толерантным (Short).mp3», продолжительностью 3 мин. 25 сек. и размером 5,49 мб. (5 764 283 байта)

Видеофайл «Бабушка отматерила чурку» продолжительностью 1 мин., начинающийся со слов «Я тебя ведь зарежу....» и заканчивающийся словами «... убью» 
...
```

Чтобы блокировать их в Интернете, нужно извлекать из записей ключевую информацию, например:
```
[[Видеоматериал]] [[«Слава РУСИ»]], размещенный на интернет-сайте, имеющий электронный адрес [[http://vkontakte.ru]] (решение [[Промышленного районного суда]] г. Курска от [[04.03.2013]]);
{
  "items": [
    {
      "type": "видеоматериал",
      "titles": [
        "«Слава РУСИ»"
      ],
      "urls": [
        "http://vkontakte.ru"
      ]
    }
  ],
  "decision": {
    "sud": {
      "name": "промышленный",
      "type": "районный"
    },
    "date": {
      "year": 2013,
      "month": 3,
      "day": 4
    }
  }
}

```

В этом примере показано, как с помощью Yargy-парсера приводить записи ФСЭМ к структурированному виду. Качество получается не идеальное, но и цель этого примера не получить продакш решение. При желании можно доработать правила, улучшить результат.

# Data

In [1]:
from random import seed, sample


def load_lines(path):
    with open(path) as file:
        for line in file:
            yield line.rstrip('\n')

            
def make_translation_table(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation_table(
    '‑–—−',
    '----'
)


def normalize_text(text):
    return text.translate(DASHES_TRANSLATION)


texts = [
    normalize_text(_)
    for _ in load_lines('texts.txt')
]

# Grammar

In [2]:
import re
import json

from ipymarkup import show_span_ascii_markup as show_markup

from yargy import (
    Parser,
    rule, or_,
    not_, and_
)
from yargy.predicates import (
    eq, type, caseless, in_, in_caseless,
    gte, lte, length_eq,
    is_capitalized, normalized,
    dictionary, gram,
)
from yargy.pipelines import (
    caseless_pipeline,
    morph_pipeline
)
from yargy.interpretation import (
    fact,
    attribute
)
from yargy import interpretation as interp
from yargy.relations import gnc_relation
from yargy.tokenizer import (
    QUOTES,
    LEFT_QUOTES,
    RIGHT_QUOTES,

    MorphTokenizer,
    TokenRule
)


def show_json(data):
    print(json.dumps(data, indent=2, ensure_ascii=False))


def test_interpretation(rule, *tests):
    parser = Parser(rule)
    for line, etalon in tests:
        match = parser.match(line)
        assert match, line
        guess = match.fact
        assert etalon == guess, guess
        

def test_match(rule, *tests):
    parser = Parser(rule)
    for line in tests:
        match = parser.match(line)
        assert match, line

## Common

In [3]:
INT = type('INT')
LATIN = type('LATIN')
DOT = eq('.')
DASH = eq('-')

## Type

In [4]:
MAIN_TYPE = morph_pipeline([
    'dvd-диск',
    'альманах',
    'аудиовизуальный материал',
    'аудиозапись',
    'аудиокомпозиция',
    'аудиоматериал',
    'аудиофайл',
    'брошюра',
    'вестник',
    'видео-файл',
    'видеозапись',
    'видеоклип',
    'видеоматериал',
    'видеообращение',
    'видеоролик',
    'видеофайл',
    'видео файл',
    'видеофильм',
    'видеофонограмма',
    'визуальный материал',
    'выпуск газеты',
    'выпуск листовки',
    'высказывания',
    'газета',
    'графическая работа',
    'графическое изображение',
    'графический файл',
    'демотиватор',
    'документ',
    'еженедельник',
    'журнал',
    'журнал-газета',
    'издание',
    'изображение',
    'информация',
    'информационное видео',
    'информационный материал',
    'кинофильм',
    'книга',
    'компьютерная игра',
    'комментарий',
    'листовка',
    'лозунг',
    'материал',
    'музыкальный альбом',
    'музыкальная композиция',
    'музыкальное произведение',
    'обозрение',
    'печатное издание',
    'печатный материал',
    'печатная продукция',
    'повесть',
    'программное обеспечение',
    'произведение',
    'прокламация',
    'публикация',
    'рисунок',
    'статья',
    'стихотворение',
    'текст аудиозаписи',
    'текст песни',
    'текстово-графическое изображение',
    'текстовый документ',
    'текстовая информация',
    'текстовая информация-статус',
    'текстовое обращение',
    'файл',
    'фильм',
    'фотография',
    'фотоизображение',
    'электронный дневник',
    'эссе',
]).interpretation(
    interp.normalized()
)

## Title

In [5]:
QUOTE = in_(QUOTES)
LEFT_QUOTE = in_(LEFT_QUOTES)
RIGHT_QUOTE = in_(RIGHT_QUOTES)

TITLE = or_(
    rule(
        LEFT_QUOTE,
        not_(RIGHT_QUOTE).repeatable(),
        RIGHT_QUOTE,
    ),
    rule(
        and_(
            QUOTE,
            not_(RIGHT_QUOTE)
        ),
        not_(QUOTE).repeatable(),
        and_(
            QUOTE,
            not_(LEFT_QUOTE)
        )
    )
)

test_match(
    TITLE,
    '«Сознание «Аль-Ваъй»',
    '"Это должен знать Русский»',
    '"Правоохранительные органы РФ фальсифицируют факты и лживо обвиняют Хизб-ут-Тахрир аль-Ислами"',
    # «Вся «правда» о жидо бандеровцах!!!!!!»
    # » ООО фирма «
)

## Url

In [6]:
URL_PATTERN = r'(?:http|https|ftp|www|vk|vkontakte|id)[a-z0-9:/\.?=\-%&_#\[\]]+'
URL_RULE = TokenRule('URL', URL_PATTERN)

URL = rule(type('URL'))

## Name

In [7]:
Name = fact(
    'Name',
    ['first', 'middle', 'last']
)


gnc = gnc_relation()

LAST = and_(
    type('RU'),
    is_capitalized()
).interpretation(
    Name.last.inflected()
).match(gnc)

FIRST = gram('Name').interpretation(
    Name.first.inflected()
).match(gnc)

MIDDLE = gram('Patr').interpretation(
    Name.middle.inflected()
).match(gnc)

ABBR = and_(
    length_eq(1),
    is_capitalized()
)

FIRST_ABBR = ABBR.interpretation(
    Name.first.custom(str.lower)
)

MIDDLE_ABBR = ABBR.interpretation(
    Name.middle.custom(str.lower)
)


NAME = or_(
    rule(
        LAST,
        FIRST_ABBR, DOT,
        MIDDLE_ABBR, DOT
    ),
    rule(
        FIRST_ABBR, DOT,
        MIDDLE_ABBR, DOT,
        LAST
    ),
    rule(
        LAST,
        FIRST_ABBR, DOT
    ),
    rule(
        FIRST_ABBR, DOT,
        LAST
    ),
    rule(
        FIRST,
        MIDDLE,
        LAST
    ),
    rule(
        LAST,
        FIRST,
        MIDDLE
    ),
    rule(
        FIRST,
        MIDDLE
    ),
).interpretation(
    Name
)


test_interpretation(
    NAME,
    ['Вострягов В.А.', Name(first='в', middle='а', last='востряг')],
    ['Т.Н. Галимова', Name(first='т', middle='н', last='галимова')],
    ['Е. Сорокоумовой', Name(first='е', last='сорокоумова')],
    ['Сергея Владимировича Наумова', Name(first='сергей', middle='владимирович', last='наумов')],
    ['Петров Константин Павлович', Name(first='константин', middle='павлович', last='петров')],
    ['Волкова Владимира Игоревича', Name(first='владимир', middle='игоревич', last='волков')],
    ['Константин Павлович', Name(first='константин', middle='павлович')],
    ['Ирину Васильевну', Name(first='ирина', middle='васильевна')]
)

## Author

In [8]:
Author = fact(
    'Author',
    ['type', 'name']
)


TYPE = morph_pipeline([
    'автор',
    'пользователь',
    'музыкальная группа',
    'группа',
    'исполнитель',
    'движение',
    'издательство',
]).interpretation(
    Author.type.normalized()
)

AUTHOR = or_(
    NAME,
    TITLE
).interpretation(
    Author.name
)

AUTHOR = rule(
    TYPE.optional(),
    AUTHOR
).interpretation(
    Author
)


test_interpretation(
    AUTHOR,
    ['музыкальной группы «Циклон Б»', Author(type='музыкальная группа', name='«Циклон Б»')],
    [
        'автор Петров Константин Павлович',
        Author(type='автор', name=Name(first='константин', middle='павлович', last='петров'))
    ],
    ['Волкова Владимира Игоревича', Author(name=Name(first='владимир', middle='игоревич', last='волков'))]
)

## Number

In [9]:
Number = fact(
    'Number',
    ['value']
)


PREFIX = or_(
    rule('№'),
    rule('№', '№'),
    rule(normalized('номер'))
)

VALUE = or_(
    rule(INT),
    rule(
        INT,
        DASH,
        INT
    ),
    rule(
        INT, '(', INT, ')'
    )
).interpretation(
    Number.value
)

NUMBER = rule(
    PREFIX,
    VALUE
).interpretation(
    Number
)

test_match(
    NUMBER,
    '№ 154',
    '№ 1-3',
    '№ 1 - 3',
    'номер 3(2)'
)

## Pages

In [10]:
Pages = fact(
    'Page',
    ['value']
)


VALUE = INT.interpretation(
    Pages.value
)

SUFFIX = rule(
    caseless('с'),
    DOT
)

PAGES = rule(
    VALUE,
    SUFFIX
).interpretation(
    Pages
)

test_match(
    PAGES,
    '416 с.'
)

## Size

In [11]:
Size = fact(
    'Size',
    ['value', 'measure']
)


def normalize_float(value):
    return float(value.replace(',', '.'))


FLOAT = rule(
    INT,
    in_({',', '.'}),
    INT
).interpretation(
    interp.custom(normalize_float)
)


def remove_spaces(value):
    return re.sub('\s+', '', value)


def normalize_separated_int(value):
    return int(remove_spaces(value))


SEPARATED_INT = or_(
    rule(INT),
    rule(INT, INT),
    rule(INT, INT, INT)
).interpretation(
    interp.custom(normalize_separated_int)
)

VALUE = or_(
    FLOAT,
    SEPARATED_INT
).interpretation(
    Size.value
)

MEASURES = {
    'мб': 'MB',
    'кб': 'KB',
    'байт': 'B'
}

MEASURE = dictionary(MEASURES).interpretation(
    Size.measure.normalized().custom(MEASURES.get)
)

SIZE = rule(
    VALUE,
    MEASURE
).interpretation(
    Size
)


test_interpretation(
    SIZE,
    ['364 МБ', Size(value=364, measure='MB')],
    ['6,12 Мб', Size(value=6.12, measure='MB')],
    ['14 000 000 кб', Size(value=14000000, measure='KB')],
)

## Duration

In [12]:
Duration = fact(
    'Duration',
    ['hours', 'minutes', 'seconds']
)


HOURS = INT.interpretation(
    Duration.hours.custom(int)
)

HOUR_WORDS = normalized('час')

MINUTES = and_(
    INT,
    gte(0),
    lte(59)
).interpretation(
    Duration.minutes.custom(int)
)

MINUTE_WORDS = or_(
    rule(normalized('минута')),
    rule(caseless('мин'), DOT.optional())
)

SECONDS = and_(
    INT,
    gte(0),
    lte(59)
).interpretation(
    Duration.seconds.custom(int)
)

SECOND_WORDS = or_(
    rule(normalized('секунда')),
    rule(caseless('сек'), DOT.optional()),
)

DURATION = rule(
    rule(
        HOURS,
        HOUR_WORDS
    ).optional(),
    rule(
        MINUTES,
        MINUTE_WORDS
    ).optional(),
    rule(
        SECONDS,
        SECOND_WORDS
    )
).interpretation(
    Duration
)


test_interpretation(
    DURATION,
    ['50 секунд', Duration(seconds=50)],
    ['02 час 11 мин. 50 сек.', Duration(hours=2, minutes=11, seconds=50)],
    ['1 час  3  минуты  24 секунды', Duration(hours=1, minutes=3, seconds=24)],
    ['3 минуты 23 секунды', Duration(minutes=3, seconds=23)],
    ['05 мин. 57 сек.', Duration(minutes=5, seconds=57)]
)

## Source

In [13]:
Source = fact(
    'Source',
    ['type', 'value']
)


TYPE = morph_pipeline([
    'cd-r диск',
    'dvd диск',
    'двд-диск',
    'компакт-диске',
    'папка',
    'журнал',
    'сборник',

    'сеть',
    'ресурс',
    'сайт',
]).interpretation(
    Source.type.normalized()
)

VALUE = or_(
    NUMBER,
    TITLE
).interpretation(
    Source.value
)

SOURCE = rule(
    TYPE,
    VALUE,
).interpretation(
    Source
)


test_interpretation(
    SOURCE,
    ['DVD диске № 23', Source(type='dvd диск', value=Number(value='23'))],
    ['папка «Указы»', Source(type='папка', value='«Указы»')],
    ['сети «Интернет»', Source(type='сеть', value='«Интернет»')]
)

## Date

In [14]:
Date = fact(
    'Date',
    ['year', 'month', 'day']
)


MONTHS = {
    'январь': 1,
    'февраль': 2,
    'март': 3,
    'апрель': 4,
    'май': 5,
    'июнь': 6,
    'июль': 7,
    'август': 8,
    'сентябрь': 9,
    'октябрь': 10,
    'ноябрь': 11,
    'декабрь': 12,
}

MONTH_NAME = dictionary(MONTHS).interpretation(
    Date.month.normalized().custom(MONTHS.get)
)

MONTH = and_(
    INT,
    gte(1),
    lte(12)
).interpretation(
    Date.month.custom(int)
)

YEAR = and_(
    INT,
    gte(1000),
    lte(3000)
).interpretation(
    Date.year.custom(int)
)

YEAR_SUFFIX = rule(
    or_(
        eq('г'),
        normalized('год')
    ),
    DOT.optional()
)

DAY = and_(
    INT,
    gte(1),
    lte(31)
).interpretation(
    Date.day.custom(int)
)

DATE = or_(
    rule(
        YEAR,
        YEAR_SUFFIX
    ),
    rule(
        MONTH_NAME,
        YEAR
    ),
    rule(
        DAY,
        DOT,
        MONTH,
        DOT,
        YEAR
    ),
    rule(
        DAY,
        MONTH_NAME,
        YEAR
    ),

)

SUFFIX = normalized('издание')

DATE = rule(
    DATE,
    SUFFIX.optional()
).interpretation(
    Date
)

test_interpretation(
    DATE,
    ['2007г. издания', Date(year=2007)],
    ['июнь 2005', Date(year=2005, month=6)],
    ['12.01.2004', Date(year=2004, month=1, day=12)],
    ['31 декабря 1990', Date(year=1990, month=12, day=31)]
)

## Sud

In [15]:
Sud = fact(
    'Sud',
    ['name', 'type']
)


CAPITALIZED = is_capitalized()

GEO = or_(
    rule(CAPITALIZED),
    rule(
        CAPITALIZED,
        DASH.optional(),
        CAPITALIZED
    )
)

NUMERAL = rule(INT)

NAME = or_(
    GEO,
    NUMERAL,
).interpretation(
    Sud.name.normalized()
)

TYPE = dictionary({
    'федеральный',
    'областной',
    'городской',
    'районный',
    'гарнизонный',
    'военный'
})

TYPE = or_(
    rule(TYPE),
    rule(TYPE, TYPE)
).interpretation(
    Sud.type.normalized()
)

SUD = rule(
    NAME.optional(),
    TYPE.optional(),
    normalized('суд')
).interpretation(
    Sud
)


test_interpretation(
    SUD,
    ['Тверского районного суда', Sud(name='тверской', type='районный')],
    ['Бугурусланского городского суда', Sud(name='бугурусланский', type='городской')],
    ['Вытегорского федерального районного суда', Sud(name='вытегорский', type='федеральный районный')],
    ['109 гарнизонного военного суда', Sud(name='109', type='гарнизонный военный')],
    ['Тверского суда', Sud(name='тверской')],
    ['Федерального суда', Sud(type='федеральный')]
)

## Decision

In [16]:
Decision = fact(
    'Decision',
    ['sud', 'date']
)


DECISION = rule(
    SUD.interpretation(
        Decision.sud
    ),
    DATE.interpretation(
        Decision.date
    )
).interpretation(
    Decision
)

## Fsem

In [17]:
Item = fact(
    'Item',
    ['type',
     attribute('titles').repeatable(),
     attribute('urls').repeatable(),
     'author', 'number', 'source',
     'pages', 'duration', 'size',
     'date'
    ]
)
Fsem = fact(
    'Fsem',
    [attribute('items').repeatable(), 'decision']
)


ATTRIBUTE = or_(
    TITLE.interpretation(Item.titles),
    URL.interpretation(Item.urls),
    AUTHOR.interpretation(Item.author),
    NUMBER.interpretation(Item.number),
    PAGES.interpretation(Item.pages),
    SIZE.interpretation(Item.size),
    DURATION.interpretation(Item.duration),
    SOURCE.interpretation(Item.source),
    DATE.interpretation(Item.date)
)

ITEM = rule(
    MAIN_TYPE.interpretation(Item.type),
    ATTRIBUTE.repeatable()
).interpretation(
    Item
)

FSEM = rule(
    ITEM.interpretation(
        Fsem.items
    ).repeatable(),
    DECISION.interpretation(
        Fsem.decision
    )
).interpretation(
    Fsem
)

## Extractor

In [18]:
def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


class Match(object):
    def __init__(self, fact, spans):
        self.fact = fact
        self.spans = spans


DEBUG = or_(
    MAIN_TYPE,
    TITLE,
    URL,
    AUTHOR,
    NUMBER,
    PAGES,
    SIZE,
    DURATION,
    SOURCE,
    DATE,
    SUD
)


TOKENIZER = MorphTokenizer().add_rules(URL_RULE)


class Extractor(object):
    def __init__(self):
        self.debug = Parser(DEBUG, tokenizer=TOKENIZER)
        self.parser = Parser(FSEM, tokenizer=TOKENIZER)

    def __call__(self, text):
        matches = self.debug.findall(text)
        spans = [_.span for _ in matches]

        line = join_spans(text, spans)
        matches = list(self.parser.findall(line))
        fact = None
        if matches:
            match = matches[0]
            fact = match.fact

        return Match(fact, spans)

    
extractor = Extractor()

# 15 ошибок, 10 полуошибок из 100
seed(11)
for text in sample(texts, 10):
    match = extractor(text)
    show_markup(text, match.spans)
    if match.fact:
        show_json(match.fact.as_json)

Книга «Справочник русского человека» автор Иванов Алексей Аркадьевич, 
───── ────────────────────────────── ───────────────────────────────  
издательство «ФЕРИ-В», отпечатанная в ООО «Типография ИПО Профиздат», 
─────────────────────                     ──────────────────────────  
2004 г, г. Москва., Крутицкий вал., д. 18 (решение Лефортовского 
──────                                             ──────────────
районного суда г. Москвы от 15.10.2012);
──────────────              ──────────  
{
  "items": [
    {
      "type": "книга",
      "titles": [
        "«Справочник русского человека»",
        "«Типография ИПО Профиздат»"
      ],
      "urls": [],
      "author": {
        "type": "издательство",
        "name": "«ФЕРИ-В»"
      },
      "date": {
        "year": 2004
      }
    }
  ],
  "decision": {
    "sud": {
      "name": "лефортовский",
      "type": "районный"
    },
    "date": {
      "year": 2012,
      "month": 10,
      "day": 15
    }
  }
}
Видеозапись «Проект 